### 观察训练集中orderHistory_train中orderType全为0的用户有多少转换为了orderFuture_train中orderType=1，同样orderHistory_train中orderType购买过1的用户有多少转换为了orderFuture_train中orderType=0；


orderHistory_train中orderType全为0的用户orderHistory_orderType_all_zero有8892人，有1133人再orderFuture_train中选择了orderType=1；剩下的7759人出现在了orderFuture_train且订单类型全为0；
这1133人中为什么会选择精品服务，看他们的rating和action；

#### 导入包和数据集

In [30]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2018-01-02 12:19:53
# @Author  : guanglinzhou (xdzgl812@163.com)

import os
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

################
print("start processing traindata")
project_path = '/Users/guanglinzhou/Desktop/DC_Mac/DC_Code/'
traindata_path = project_path + "data/train/"
traindata_output_path = project_path + "preprocess/"
action_train = pd.read_csv(traindata_path + "action_train.csv")
orderFuture_train = pd.read_csv(traindata_path + "orderFuture_train.csv")
orderHistory_train = pd.read_csv(traindata_path + "orderHistory_train.csv")
userComment_train = pd.read_csv(traindata_path + "userComment_train.csv")
userProfile_train = pd.read_csv(traindata_path + "userProfile_train.csv")

start processing traindata


In [38]:
zz=action_train.groupby('userid').count()
zz['num']=zz['actionType']
del zz['actionType']
del zz['actionTime']
zz.reset_index(inplace=True)
userid_arr=zz[zz['num']<5]['userid'].values
cnt=0
print('Future_train_shape: '+str(orderFuture_train.shape[0]))
print('fture_type1的数量： '+str(orderFuture_train[orderFuture_train['orderType']==1].shape[0]))
for userid in userid_arr:
    if(orderFuture_train[orderFuture_train['userid']==userid]['orderType'].values[0]==1):
        cnt+=1
print('actionType_num<5:  '+str(len(userid_arr)))
print('actionType_num<5 且fture_type1的数量: '+str(cnt))
cnt2=0

for userid in  userid_arr:
    if(userid not in orderHistory_train['userid'].values and orderFuture_train[orderFuture_train['userid']==userid]['orderType'].values[0]==1):
        cnt2+=1
print('actionType_num<5 且没有history_order 且 fture_type1的数量:'+str(cnt2))

Future_train_shape: 40307
fture_type1的数量： 6625
actionType_num<5:  4490
actionType_num<5 且fture_type1的数量: 466
actionType_num<5 且没有history_order 且 fture_type1的数量:424


In [37]:
testdata_path = project_path + "data/test/"
action_test = pd.read_csv(testdata_path + "action_test.csv")
orderFuture_test=pd.read_csv(testdata_path + 'orderFuture_test.csv')
zz=action_test.groupby('userid').count()
zz['num']=zz['actionType']
del zz['actionType']
del zz['actionTime']
zz.reset_index(inplace=True)
userid_arr=zz[zz['num']<5]['userid'].values
print('Future_test_shape: '+str(orderFuture_test.shape[0]))
print('actionType_num<5:  '+str(len(userid_arr)))

Future_test_shape: 10076
actionType_num<5:  1122


In [4]:
orderHistory_orderType_all_zero=pd.DataFrame(columns=[['userid','orderid','orderTime','orderType','city','country','continent']])
orderHistory_orderType_all_zero_num=0
def fun_construct_orderHistory_orderType_all_zero(df):
    global orderHistory_orderType_all_zero
    global orderHistory_orderType_all_zero_num
    if(1 not in df['orderType'].values):
        if(orderHistory_orderType_all_zero_num!=0): #由于groupby()会传递第一组数据两次，所以跳过第一次     
            orderHistory_orderType_all_zero=pd.concat([orderHistory_orderType_all_zero,df],axis=0)
    orderHistory_orderType_all_zero_num+=1

In [5]:
orderHistory_train.groupby('userid').apply(fun_construct_orderHistory_orderType_all_zero)

""


orderHistory_train中orderType全为0的用户:orderHistory_orderType_all_zero

In [6]:
orderHistory_orderType_all_zero

,userid,orderid,orderTime,orderType,city,country,continent
0,100000000013,1000015,1481714516,0,柏林,德国,欧洲
1,100000000013,1000014,1501959643,0,旧金山,美国,北美洲
2,100000000393,1000033,1499440296,0,巴黎,法国,欧洲
3,100000000459,1000036,1480601668,0,纽约,美国,北美洲
4,100000000459,1000034,1479146723,0,巴厘岛,印度尼西亚,亚洲
5,100000000637,1000040,1486739692,0,巴塞罗那,西班牙,欧洲
6,100000000695,1000042,1498751734,0,东京,日本,亚洲
7,100000000949,1000050,1500852591,0,珀斯,澳大利亚,大洋洲
8,100000000949,1000051,1500079241,0,珀斯,澳大利亚,大洋洲
9,100000000949,1000047,1500493418,0,珀斯,澳大利亚,大洋洲


orderFuture_train中orderType为1的用户：orderfuture_orderType_one

In [71]:
orderfuture_orderType_one=orderFuture_train[orderFuture_train['orderType']==1]

In [72]:
orderfuture_orderType_one.head()

,userid,orderType
10,100000000637,1
11,100000000695,1
14,100000000867,1
16,100000000975,1
17,100000001023,1


训练集中orderHistory_train中orderType全为0的用户有1133个人转换为orderFuture_train中orderType=1的userid列表

In [73]:
userid_list_of_history0_future1=list(set(orderHistory_orderType_all_zero['userid']) & set(orderfuture_orderType_one['userid']))

In [74]:
len(userid_list_of_history0_future1)

1133

In [11]:
len(set(orderHistory_orderType_all_zero['userid']))

8892

In [12]:
userid_list_of_history0_future0=list(set(orderHistory_orderType_all_zero['userid'])-set(userid_list_of_history0_future1))

In [13]:
len(userid_list_of_history0_future0)

7759

确认，这7759个userid在orderFuture中orderType都为0

In [14]:
userid_list_of_history0_future0_num=0
for userid in userid_list_of_history0_future0:
    if(orderFuture_train[orderFuture_train['userid']==userid]['orderType'].values[0]!=0):
        userid_list_of_history0_future0_num+=1
print(userid_list_of_history0_future0_num)

0


查看这1133个人的评分情况

userComment_train中userid不重复

In [15]:
userComment_train.shape[0]-len(set(userComment_train['userid']))

0

In [16]:
rating_userid_dict_history0_future1=defaultdict(lambda:[])
for userid in userid_list_of_history0_future1:
    if(userid in userComment_train['userid'].values):
        Comment_rating=userComment_train[userComment_train['userid']==userid]['rating']
        rating_userid_dict_history0_future1[Comment_rating.values[0]].append(userid)
    else:
        rating_userid_dict_history0_future1['NaN'].append(userid)
print(rating_userid_dict_history0_future1.keys())

for rating in rating_userid_dict_history0_future1.keys():
    print('rating '+str(rating)+' has '+str(len(rating_userid_dict_history0_future1[rating]))+' userid')

dict_keys([5.0, 4.0, 'NaN', 3.0, 1.0, 2.0, 4.3300000000000001, 3.6699999999999999])
rating 5.0 has 724 userid
rating 4.0 has 19 userid
rating NaN has 375 userid
rating 3.0 has 6 userid
rating 1.0 has 4 userid
rating 2.0 has 3 userid
rating 4.33 has 1 userid
rating 3.67 has 1 userid


从上可看出1133个人中，如果评论记录中有当前的userid，且评分>3分，占据了2/3的比例。那这不能代表历史Type=0，rating>3，future_orderType就为1，所以我要看一下rating_userid_dict_history0_future0的情况

In [17]:
rating_userid_dict_history0_future0=defaultdict(lambda:[])
for userid in userid_list_of_history0_future0:
    if(userid in userComment_train['userid'].values):
        Comment_rating=userComment_train[userComment_train['userid']==userid]['rating']
        rating_userid_dict_history0_future0[Comment_rating.values[0]].append(userid)
    else:
        rating_userid_dict_history0_future0['NaN'].append(userid)
print(rating_userid_dict_history0_future0.keys())

for rating in rating_userid_dict_history0_future0.keys():
    print('rating '+str(rating)+' has '+str(len(rating_userid_dict_history0_future0[rating]))+' userid')

dict_keys([5.0, 'NaN', 2.0, 4.0, 1.0, 3.0])
rating 5.0 has 4412 userid
rating NaN has 3221 userid
rating 2.0 has 10 userid
rating 4.0 has 66 userid
rating 1.0 has 24 userid
rating 3.0 has 26 userid


从中可以看出，历史Type=0，Future_Type=0的用户中，rating>3的比例占据了58%左右，所以不代表History_type=0,rationg>3，Future_Type就为1。
但是通过观察数据发现，History_Type=0的用户中，如果在提交订单后继续浏览了产品(actionType:2~4)，则Future_Type基本为1，而Future_Type=0的用户在历史提交订单后，不会再浏览产品了(actionType没有2~4)。

所以我要输出每个userid在最后一次提交历史订单后，继续浏览产品的次数(actionType:2~4).


In [61]:
history_action_merge_by_time_train=pd.read_csv(project_path+'preprocess/middle_file/history_action_merge_by_time_train.csv')

In [67]:
userid_list_history0_future1_rating_larger3=[]
userid_list_history0_future1_rating_larger3.extend(rating_userid_dict_history0_future1[5])
userid_list_history0_future1_rating_larger3.extend(rating_userid_dict_history0_future1[4])
print(len(userid_list_history0_future1_rating_larger3))

743


In [68]:
for i in userid_list_history0_future1_rating_larger3:
    print(i)

114740494336
112604565504
114807967747
114746791939
110769035269
111150813197
111150485518
111192985615
112522080286
112537065502
114747093030
110720333862
110106073127
112524683303
112525865003
112505665581
111840346157
111151048759
114713880631
110103035961
110109435962
114746294334
110100516927
110768435262
110162333763
112624265285
110120818763
110972010573
111145646155
110946517073
110107314260
110104135766
111145113690
112633765980
111803019358
111192016992
111140816993
111424516195
111445516391
114718380138
111122585710
111140616311
111499288696
111155413119
111803414657
110103818368
110104133765
111143716998
111158046857
114127982730
112520480908
111125948556
110972233869
114826653842
112529883284
114715093138
111847219350
114740291737
110173735067
111143213214
114822467743
110911117473
111805448355
112536969382
114713680039
111193516198
114743380139
114664067242
110778833068
111845546158
114793982133
112630165684
112630165687
114829553849
111124885697
114824450242
111847819457

In [52]:
userid_actionType_num_dict_history0_future1=defaultdict(lambda: defaultdict(lambda:0))
for userid in userid_list_history0_future1_rating_larger3:
#     print(userid)
    #     print(userid)
#     print(history_action_merge_by_time_train[(history_action_merge_by_time_train['userid']==userid) & (history_action_merge_by_time_train['actionTime_zero_orderTime_one']==1)].index.values)
#     print(history_action_merge_by_time_train[(history_action_merge_by_time_train['userid']==userid) & (history_action_merge_by_time_train['actionTime_zero_orderTime_one']==1)].index.values.max())
#     print(history_action_merge_by_time_train[history_action_merge_by_time_train['userid']==userid].index.values.max())
    
    last_orderTime_index=history_action_merge_by_time_train[(history_action_merge_by_time_train['userid']==userid) & (history_action_merge_by_time_train['actionTime_zero_orderTime_one']==1)].index.values.max()
    last_userid_action_index=history_action_merge_by_time_train[history_action_merge_by_time_train['userid']==userid].index.values.max()
    
    last_range_df=history_action_merge_by_time_train.iloc[last_orderTime_index+1:last_userid_action_index+1,:]
#     print(last_range_df)
    for actionType in last_range_df['actionType'].values:
        userid_actionType_num_dict_history0_future1[userid][actionType]+=1
Type234_num=0
for userid in userid_actionType_num_dict_history0_future1.keys():
    Type234_num+=userid_actionType_num_dict_history0_future1[userid][2]+userid_actionType_num_dict_history0_future1[userid][3]+userid_actionType_num_dict_history0_future1[userid][4]
print("在用户提交最后一次订单后，继续浏览商品的数量为："+Type234_num)

defaultdict(<function <lambda> at 0x1096a7ae8>, {114740494336: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9488>, {1.0: 6}), 112604565504: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9e18>, {1.0: 57}), 114807967747: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9d08>, {1.0: 32}), 114746791939: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9b70>, {1.0: 12}), 110769035269: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9ae8>, {5.0: 20}), 111150813197: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9a60>, {1.0: 21}), 111150485518: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b99d8>, {1.0: 14}), 111192985615: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9950>, {5.0: 49}), 112522080286: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9730>, {1.0: 8}), 112537065502: defaultdict(<function <lambda>.<locals>.<lambda> at 0x1098b9620>, {1.0: 63}), 114747093030: defaultdict(<functi

In [53]:
userid_actionType_num_dict[114740494336]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>>, {1.0: 6})

In [40]:
index_range=history_action_merge_by_time_train[history_action_merge_by_time_train['userid']==114740494336].index.values.max()-history_action_merge_by_time_train[(history_action_merge_by_time_train['userid']==114740494336) & (history_action_merge_by_time_train['actionTime_zero_orderTime_one']==1)].index.values.max() 

In [24]:
index_range

6

In [ ]:
last_orderTime_index=history_action_merge_by_time_train[(history_action_merge_by_time_train['userid']==userid) & (history_action_merge_by_time_train['actionTime_zero_orderTime_one']==1)].index.values.max()

In [104]:
userComment_train = pd.read_csv(traindata_path + "userComment_train.csv")
userComment_train=userComment_train.fillna(0)

In [110]:
len(str(userComment_train.iloc[0,3]).split('|'))

1

In [106]:
if(userComment_train.iloc[0,3]==0):
    print('aaa')

aaa


In [107]:
type(userComment_train.iloc[0,3])

int

In [117]:
userComment_train.iloc[6,4]

"['真心','好','惊喜','惊喜','合理','还','真心','赞']"

In [111]:
print('constructing feature tags_num and keywords_num')
userComment_train = pd.read_csv(traindata_path + "userComment_train.csv")
userComment_train=userComment_train.fillna(0)
userid_tags_keywords_num_train=open(feature_file_path+'userid_tags_keywords_num_train.csv')
userid_tags_keywords_num_train.write('userid,tages_num,keywords_num')
for index in userComment_train.index:
    write_userid=userComment_train.iloc[index,0]
    if(userComment_train.iloc[index,3]!=0):
        write_tages_num=len(userComment_train.iloc[index,3].split('|'))
    else:
        write_tages_num=0
    if(userComment_train.iloc[index,4]!=0):
        write_keywords_num=len(userComment_train.iloc[index,4].split('|'))
    else:
        write_keywords_num=0
    userid_tags_keywords_num_train.write('\n')
    userid_tags_keywords_num_train.write(str(write_userid)+','+str(write_tages_num)+','+str(write_keywords_num))

constructing feature tags_num and keywords_num


NameError: name 'feature_file_path' is not defined

In [118]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2017-12-29 12:19:53
# @Author  : guanglinzhou (xdzgl812@163.com)

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

# 特征均可通过merge操作，on='userid'来合并到表xxxdata_base.csv中

###########-----------------------读表----------------------------###########
project_path = '/Users/guanglinzhou/Desktop/DC_Mac/DC_Code/'
traindata_path = project_path + "data/train/"
traindata_path = project_path + "data/train/"
testdata_path = project_path + "data/test/"
feature_file_path = project_path + 'preprocess/feature_file/feature_from_jingsaiquan/'

In [119]:
# 用户点击X与总点击量比值

action_train = pd.read_csv(traindata_path + "action_train.csv")
action_test = pd.read_csv(testdata_path + "action_test.csv")


def fun_user_X_click_rate_train(df):
    global num_click_rate
    global userid_click_X_rate_train
    global click1_num_train
    global click2_num_train
    global click3_num_train
    global click4_num_train
    global click5_num_train
    global click6_num_train
    global click7_num_train
    global click8_num_train
    global click9_num_train

    if (num_click_rate != 0):
        userid = df['userid'].values[0]
        if (1 in df['actionType'].values):
            click1_rate = df[df['actionType'] == 1].shape[0] / click1_num_train
        else:
            click1_rate = np.nan
        if (2 in df['actionType'].values):
            click2_rate = df[df['actionType'] == 2].shape[0] / click2_num_train
        else:
            click2_rate = np.nan
        if (3 in df['actionType'].values):
            click3_rate = df[df['actionType'] == 3].shape[0] / click3_num_train
        else:
            click3_rate = np.nan
        if (4 in df['actionType'].values):
            click4_rate = df[df['actionType'] == 4].shape[0] / click4_num_train
        else:
            click4_rate = np.nan
        if (5 in df['actionType'].values):
            click5_rate = df[df['actionType'] == 5].shape[0] / click5_num_train
        else:
            click5_rate = np.nan
        if (6 in df['actionType'].values):
            click6_rate = df[df['actionType'] == 6].shape[0] / click6_num_train
        else:
            click6_rate = np.nan
        if (7 in df['actionType'].values):
            click7_rate = df[df['actionType'] == 7].shape[0] / click7_num_train
        else:
            click7_rate = np.nan
        if (8 in df['actionType'].values):
            click8_rate = df[df['actionType'] == 8].shape[0] / click8_num_train
        else:
            click8_rate = np.nan
        if (9 in df['actionType'].values):
            click9_rate = df[df['actionType'] == 9].shape[0] / click9_num_train
        else:
            click9_rate = np.nan
        userid_click_X_rate_train[userid]['click1_rate_train'] = click1_rate
        userid_click_X_rate_train[userid]['click2_rate_train'] = click2_rate
        userid_click_X_rate_train[userid]['click3_rate_train'] = click3_rate
        userid_click_X_rate_train[userid]['click4_rate_train'] = click4_rate
        userid_click_X_rate_train[userid]['click5_rate_train'] = click5_rate
        userid_click_X_rate_train[userid]['click6_rate_train'] = click6_rate
        userid_click_X_rate_train[userid]['click7_rate_train'] = click7_rate
        userid_click_X_rate_train[userid]['click8_rate_train'] = click8_rate
        userid_click_X_rate_train[userid]['click9_rate_train'] = click9_rate

    num_click_rate += 1
    return df


click1_num_train = action_train[action_train['actionType'] == 1].shape[0]
click2_num_train = action_train[action_train['actionType'] == 2].shape[0]
click3_num_train = action_train[action_train['actionType'] == 3].shape[0]
click4_num_train = action_train[action_train['actionType'] == 4].shape[0]
click5_num_train = action_train[action_train['actionType'] == 5].shape[0]
click6_num_train = action_train[action_train['actionType'] == 6].shape[0]
click7_num_train = action_train[action_train['actionType'] == 7].shape[0]
click8_num_train = action_train[action_train['actionType'] == 8].shape[0]
click9_num_train = action_train[action_train['actionType'] == 9].shape[0]
num_click_rate = 0
userid_click_X_rate_train = defaultdict(lambda: defaultdict(lambda: 0))
action_train.groupby('userid').apply(fun_user_X_click_rate_train)
clickX_rate_df = pd.DataFrame.from_dict(userid_click_X_rate_train, orient='index').reset_index()
clickX_rate_df.rename(columns={'index': 'userid'})

,userid,click1_rate_train,click2_rate_train,click3_rate_train,click4_rate_train,click5_rate_train,click6_rate_train,click7_rate_train,click8_rate_train,click9_rate_train
0,100000000013,0.000205,0.000022,0.000051,0.000022,0.000067,0.000079,0.000036,0.000139,0.000163
1,100000000111,0.000003,NaN,NaN,NaN,0.000002,0.000004,NaN,NaN,NaN
2,100000000127,0.000005,NaN,NaN,NaN,0.000004,NaN,0.000071,NaN,NaN
3,100000000231,0.000039,0.000134,0.000026,0.000110,0.000021,0.000026,NaN,NaN,NaN
4,100000000379,0.000109,0.000067,0.000090,0.000132,0.000029,0.000048,0.000036,NaN,NaN
5,100000000393,0.000060,NaN,0.000013,NaN,0.000017,0.000013,0.000036,0.000035,0.000054
6,100000000423,0.000977,0.000557,0.000141,0.000330,0.000113,0.000062,0.000463,0.005316,0.000489
7,100000000459,0.000044,NaN,NaN,NaN,0.000040,0.000132,NaN,0.000069,0.000163
8,100000000465,0.000052,NaN,NaN,NaN,0.000017,0.000026,NaN,0.000104,0.000163
9,100000000471,0.000005,NaN,0.000038,NaN,0.000023,0.000022,NaN,0.000035,NaN
